In [1]:
#load data
import csv
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

samples = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
        
#images = []
#measurements = []
# augment the dataset with images from side cameras
#correction = [0., 0.22, -0.22]
#for line in lines:
#    steering_center = float(line[3]);
#    for idx1 in range(3):
#        source_path = line[idx1]
#        file_name = source_path.split('/')[-1]
#        current_path = '../data/IMG/' + file_name
#        image = cv2.imread(current_path)
#        images.extend(image)
#        measurements.append(steering_center + correction[idx1])

CORRECTION = [0., 0.22, -0.22]
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                center_angle = float(batch_sample[3])
                for idx1 in range(3):
                    name = './data/IMG/' + batch_sample[idx1].split('/')[-1]
                    image = cv2.imread(name)
                    angle = center_angle + CORRECTION[idx1]
                    images.append(image)
                    angles.append(angle)
                    #augment by flipping the image and angle
                    images.append(np.fliplr(image))
                    angles.append(-1. * angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

BATCH_SIZE = 128
train_generator = generator(train_samples, batch_size = BATCH_SIZE)
validation_generator = generator(validation_samples, batch_size= BATCH_SIZE)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Convolution2D, Cropping2D

#Build the model following the architecture from NVIDIA's paper
model = Sequential()
model.add(Lambda(lambda x : x/127.5 - 1., input_shape = (160, 320, 3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Convolution2D(24, 5, 5, subsample = (2,2), activation = 'relu'))
model.add(Convolution2D(36, 5, 5, subsample = (2,2), activation = 'relu'))
model.add(Convolution2D(48, 5, 5, subsample = (2,2), activation = 'relu'))
model.add(Convolution2D(64, 3, 3, activation = 'relu'))
model.add(Convolution2D(64, 3, 3, activation = 'relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

#Complile and train the model
model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= /
            len(train_samples), validation_data=validation_generator, /
            nb_val_samples=len(validation_samples), nb_epoch=5)
model.save('model.h5')
print('model saved')


    